In [1]:
import sys
import numpy as np
import pandas as pd
# from matplotlib.pyplot import imread
from skimage.io import imread
from skimage.transform import resize
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from time import time
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
import keras.layers as layers
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng
from keras.utils import plot_model
from deepcapsulelayers import Conv2DCaps, ConvCapsuleLayer3D, CapsuleLayer, CapsToScalars, Mask_CID, ConvertToCaps, FlattenCaps
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from time import time
import glob
from IPython.display import Image


Using TensorFlow backend.


### Please give below paths as per the paths in your system

In [2]:
train_folder = "data/images_background/"
val_folder = 'data/images_evaluation/'
save_path = 'data/'
train_type = 'capsule_rev1' #CONVOLUTIONAL or CAPSULE

In [3]:
def loadimgs(path,n = 0):
    '''
    path => Path of train directory or test directory
    '''
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet seperately so we can isolate them later
    for alphabet in [x for x in os.listdir(path) if not x.startswith('.')]:
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has it's own column in the array, so  load seperately
        for letter in [x for x in os.listdir(alphabet_path) if not x.startswith('.')]:
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(alphabet_path, letter)
            # read all the images in the current category
            for filename in [x for x in os.listdir(letter_path) if not x.startswith('.')]:
                image_path = os.path.join(letter_path, filename)
                image = imread(image_path)
#                 image = resize(image, (52, 52))
                category_images.append(image)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            # edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

### Loading images into tensors

In [4]:
# X,y,c=loadimgs(train_folder)
# Xval,yval,cval=loadimgs(val_folder)

### Saving the tensors on disk

In [5]:
# with open(os.path.join(save_path,"train.pickle"), "wb") as f:
#     pickle.dump((X,c),f)

# with open(os.path.join(save_path,"val.pickle"), "wb") as f:
#     pickle.dump((Xval,cval),f)

### Parameter initializers

In [6]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [7]:
def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

### Loading tensors

In [8]:
with open(os.path.join(save_path, "train.pickle"), "rb") as f:
    (Xtrain, train_classes) = pickle.load(f)
    
print("Training alphabets: \n")
print(list(train_classes.keys()))

with open(os.path.join(save_path, "val.pickle"), "rb") as f:
    (Xval, val_classes) = pickle.load(f)

print("Validation alphabets:", end="\n\n")
print(list(val_classes.keys()))

Training alphabets: 

['Blackfoot_(Canadian_Aboriginal_Syllabics)', 'Braille', 'Futurama', 'Balinese', 'Cyrillic', 'Latin', 'Greek', 'Bengali', 'Grantha', 'Gujarati', 'Mkhedruli_(Georgian)', 'Tagalog', 'Alphabet_of_the_Magi', 'Japanese_(hiragana)', 'Tifinagh', 'Sanskrit', 'Hebrew', 'N_Ko', 'Inuktitut_(Canadian_Aboriginal_Syllabics)', 'Armenian', 'Syriac_(Estrangelo)', 'Asomtavruli_(Georgian)', 'Burmese_(Myanmar)', 'Japanese_(katakana)', 'Early_Aramaic', 'Korean', 'Arcadian', 'Ojibwe_(Canadian_Aboriginal_Syllabics)', 'Anglo-Saxon_Futhorc', 'Malay_(Jawi_-_Arabic)']
Validation alphabets:

['Glagolitic', 'Kannada', 'Angelic', 'Tengwar', 'Manipuri', 'Keble', 'Aurek-Besh', 'Syriac_(Serto)', 'Atemayar_Qelisayer', 'Atlantean', 'Malayalam', 'Ge_ez', 'Oriya', 'Mongolian', 'Tibetan', 'Old_Church_Slavonic_(Cyrillic)', 'ULOG', 'Sylheti', 'Gurmukhi', 'Avesta']


### Train and test util functions

In [9]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
    
    return pairs, targets

In [10]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [11]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

### Models

In [12]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [13]:
def DeepCapsNet(input_shape, n_class, routings):
    # assemble encoder
    x = Input(shape=input_shape)
    l = x

    l = Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding="same")(l)  # common conv layer
    l = BatchNormalization()(l)
    l = ConvertToCaps()(l)

    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
#     l_skip = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
#     l = layers.Add()([l, l_skip])

    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
#     l_skip = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
#     l = layers.Add()([l, l_skip])

    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
#     l_skip = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
#     l = layers.Add()([l, l_skip])
    l1 = l

    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
#     l_skip = ConvCapsuleLayer3D(kernel_size=3, num_capsule=32, num_atoms=8, strides=1, padding='same', routings=3)(l)
    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 8, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
#     l = layers.Add()([l, l_skip])
    l2 = l

    la = FlattenCaps()(l2)
    lb = FlattenCaps()(l1)
    l = layers.Concatenate(axis=-2)([la, lb])
#     l = Dropout(0.4)(l)
    digits_caps = CapsuleLayer(num_capsule=n_class, dim_capsule=32, routings=routings, channels=0, name='digit_caps')(l)

    l = CapsToScalars(name='capsnet')(digits_caps)

    m_capsnet = Model(inputs=x, outputs=l, name='capsnet_model')
    return m_capsnet


In [14]:
def BaseCapsNet2(input_shape, n_class, routings):
    # assemble encoder
    x = Input(shape=input_shape)
    l = x

    l = Conv2D(256, (9, 9), strides=(1,12), activation='relu', padding="same")(l) 
    l = BatchNormalization()(l)
    l = ConvertToCaps()(l)
    
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(16, 6, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
    
    l = FlattenCaps()(l)
    
    digits_caps = CapsuleLayer(num_capsule=20, dim_capsule=16, routings=routings, channels=0, name='digit_caps')(l)
    l = CapsToScalars(name='capsnet')(digits_caps)

    m_capsnet = Model(inputs=x, outputs=l, name='capsnet_model')
    

    return m_capsnet

In [15]:
def BaseCapsNet1(input_shape, n_class, routings):
    # assemble encoder
    x = Input(shape=input_shape)
    l = x

    l = Conv2D(256, (9, 9), strides=(1,12), activation='relu', padding="same")(l) 
    l = BatchNormalization()(l)
    l = ConvertToCaps()(l)
    
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(32, 4, kernel_size=(3, 3), strides=(1, 1), r_num=1, b_alphas=[1, 1, 1])(l)
    l = Conv2DCaps(16, 6, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
    
    l = FlattenCaps()(l)
    
    digits_caps = CapsuleLayer(num_capsule=10, dim_capsule=8, routings=routings, channels=0, name='digit_caps')(l)
    l = CapsToScalars(name='capsnet')(digits_caps)

    m_capsnet = Model(inputs=x, outputs=l, name='capsnet_model')
    

    return m_capsnet

In [16]:
def BaseCapsNet(input_shape, n_class, routings):
    # assemble encoder
    x = Input(shape=input_shape)
    l = x

    l = Conv2D(256, (9, 9), strides=(2, 2), activation='relu', padding="same")(l) 
    l = BatchNormalization()(l)
    l = Conv2D(256, (9, 9), strides=(2, 2), activation='relu', padding="same")(l) 
    l = BatchNormalization()(l)
    l = ConvertToCaps()(l)

    l = Conv2DCaps(16, 6, kernel_size=(3, 3), strides=(2, 2), r_num=1, b_alphas=[1, 1, 1])(l)
    l = FlattenCaps()(l)
    digits_caps = CapsuleLayer(num_capsule=20, dim_capsule=32, routings=routings, channels=0, name='digit_caps')(l)
    l = CapsToScalars(name='capsnet')(digits_caps)

    m_capsnet = Model(inputs=x, outputs=l, name='capsnet_model')
    

    return m_capsnet

In [17]:
def get_capsnet_model(input_shape, n_class, routings):

    left_input = Input(shape=input_shape)
    right_input = Input(shape=input_shape)

#     input = Input(shape=input_shape)
#     # Layer 1: Just a conventional Conv2D layer
#     conv1 = Conv2D(filters=128, kernel_size=36, strides=1, padding='valid', activation='relu', name='conv1',
#                   kernel_initializer=initialize_weights, bias_initializer=initialize_bias,
#                    kernel_regularizer=l2(2e-4),)(input)

#     # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
#     primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=16, kernel_size=36, strides=2, padding='valid')

#     # Layer 3: Capsule layer. Routing algorithm works here.
#     digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
#                              name='digitcaps')(primarycaps)

#     # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
#     # If using tensorflow, this will not be necessary. :)
#     out_caps = Length(name='capsnet')(digitcaps)

    tunnel = BaseCapsNet2(input_shape, n_class, routings)
    tunnel.summary()

    encoded_l = tunnel(left_input)
    encoded_r = tunnel(right_input)

    L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(L1_distance)

    train_model = Model(inputs=[left_input, right_input], outputs=prediction)

    return train_model


### Load models

In [18]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

model_path = "checkpoints_{0}".format(train_type)

os.makedirs("checkpoints_{0}".format(train_type), exist_ok=True)
file_path = "checkpoints_{0}".format(train_type)+"/siamese-epoch-{epoch:05d}-lr-" + "-train_loss-{loss:.4f}-val_loss-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_path,
                             monitor=['loss', 'val_loss'],
                             verbose=1,
                             save_best_only=False,
                             save_weights_only=False,
                             mode='min',
                             period=5)
tensorboard = TensorBoard(log_dir="logs_{}".format(train_type)+"/{}".format(time()), histogram_freq=0)
lr_decay = LearningRateScheduler(schedule=lambda epoch: 0.001 * (0.9 ** epoch))


In [21]:
ckpts = glob.glob("checkpoints_{0}".format(train_type)+"/*.hdf5")
initial_epoch = 0

model = get_capsnet_model(input_shape=[105, 105, 1], n_class=10, routings=3)

if len(ckpts) != 0:
    latest_ckpt = max(ckpts, key=os.path.getctime)
    print("loading from checkpoint: ", latest_ckpt)
    initial_epoch = int(latest_ckpt[latest_ckpt.find("-epoch-") + len("-epoch-"):latest_ckpt.rfind("-lr-")])
    model.load_weights(latest_ckpt)

model.summary()
plot_model(model, to_file='model.png')
Image(retina=True, filename='model.png')

optimizer = Adam(lr = 0.00001)
model.compile(loss="binary_crossentropy",optimizer=optimizer)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 105, 105, 1)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 105, 9, 256)       20992     
_________________________________________________________________
batch_normalization_2 (Batch (None, 105, 9, 256)       1024      
_________________________________________________________________
convert_to_caps_2 (ConvertTo (None, 105, 9, 256, 1)    0         
_________________________________________________________________
conv2d_caps_5 (Conv2DCaps)   (None, 53, 5, 32, 4)      294912    
_________________________________________________________________
conv2d_caps_6 (Conv2DCaps)   (None, 53, 5, 32, 4)      147456    
_________________________________________________________________
conv2d_caps_7 (Conv2DCaps)   (None, 53, 5, 32, 4)      147456    
__________

In [ ]:
model.fit_generator(generator=generate(32), 
                    steps_per_epoch=100, 
                    epochs=1000,
                    initial_epoch=initial_epoch,
                    validation_data=generate(32, 'val'), 
                    validation_steps=2,
                    callbacks=[checkpoint, tensorboard],
                    use_multiprocessing=True)

Epoch 320/1000
100/100 [==============================] - 24s 237ms/step - loss: 0.2174 - val_loss: 0.3073
Epoch 321/1000
100/100 [==============================] - 21s 214ms/step - loss: 0.2086 - val_loss: 0.3468
Epoch 322/1000
100/100 [==============================] - 21s 214ms/step - loss: 0.2125 - val_loss: 0.4109
Epoch 323/1000
100/100 [==============================] - 21s 212ms/step - loss: 0.2092 - val_loss: 0.3457

Epoch 00323: saving model to checkpoints_capsule_rev1/siamese-epoch-00323-lr--train_loss-0.2092-val_loss-0.3457.hdf5
Epoch 324/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.2067 - val_loss: 0.2694
Epoch 325/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.2032 - val_loss: 0.3305
Epoch 326/1000
100/100 [==============================] - 21s 212ms/step - loss: 0.2091 - val_loss: 0.2991
Epoch 327/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.2086 - val_loss: 0.2879
Epoch 328/1000
100/100 [==

100/100 [==============================] - 21s 213ms/step - loss: 0.2064 - val_loss: 0.2864

Epoch 00383: saving model to checkpoints_capsule_rev1/siamese-epoch-00383-lr--train_loss-0.2064-val_loss-0.2864.hdf5
Epoch 384/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.2010 - val_loss: 0.2255
Epoch 385/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.2106 - val_loss: 0.2756
Epoch 386/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.1901 - val_loss: 0.2927
Epoch 387/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.2013 - val_loss: 0.2887
Epoch 388/1000
100/100 [==============================] - 21s 213ms/step - loss: 0.1998 - val_loss: 0.2483

Epoch 00388: saving model to checkpoints_capsule_rev1/siamese-epoch-00388-lr--train_loss-0.1998-val_loss-0.2483.hdf5
Epoch 389/1000
100/100 [==============================] - 21s 214ms/step - loss: 0.2094 - val_loss: 0.2805
Epoch 390/1000
100/100 [======

In [ ]:
x_test, y_test = get_batch(4)

In [ ]:
y_test

In [ ]:
prediction = model.predict(x_test)

In [ ]:
prediction

In [ ]:
layer_name = 'lambda_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(x_test)

In [ ]:
layer_name = 'model_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).get_output_at(-1))
intermediate_output = intermediate_layer_model.predict(x_test)

In [ ]:
intermediate_output

In [ ]:
print("Starting training process!")
print("-------------------------------------")
t_start = time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

### Testing

In [39]:
def nearest_neighbour_correct(pairs,targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
        given by (pairs, targets)"""
    L2_distances = np.zeros_like(targets)
    for i in range(len(targets)):
        L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2))
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0

In [40]:
def test_nn_accuracy(N_ways,n_trials):
    """Returns accuracy of NN approach """
    print("Evaluating nearest neighbour on {} unique {} way one-shot learning tasks ...".format(n_trials,N_ways))

    n_right = 0
    
    for i in range(n_trials):
        pairs,targets = make_oneshot_task(N_ways,"val")
        correct = nearest_neighbour_correct(pairs,targets)
        n_right += correct
    return 100.0 * n_right / n_trials

In [41]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [42]:
ways = np.arange(1,20,2)
resume =  False
trials = 50
val_accs, train_accs,nn_accs = [], [], []


In [43]:
for N in ways:    
    val_accs.append(test_oneshot(model, N, trials, "val", verbose=True))
#     train_accs.append(test_oneshot(model, N, trials, "train", verbose=True))
    nn_acc = test_nn_accuracy(N, trials)
    nn_accs.append(nn_acc)
    print ("NN Accuracy = ", nn_acc)
    print("---------------------------------------------------------------------------------------------------------------")

Evaluating model on 50 random 1 way one-shot learning tasks ... 

Got an average of 100.0% 1 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 1 way one-shot learning tasks ...
NN Accuracy =  100.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 3 way one-shot learning tasks ... 

Got an average of 76.0% 3 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 3 way one-shot learning tasks ...
NN Accuracy =  64.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 5 way one-shot learning tasks ... 

Got an average of 66.0% 5 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 5 way one-shot learning tasks ...
NN Accuracy =  44.0
---------------------------------------------------------------------------------------------------------------

In [ ]:
with open(os.path.join(save_path,"accuracies.pickle"), "wb") as f:
    pickle.dump((val_accs,train_accs,nn_accs),f)

### Result visualization

In [ ]:
def concat_images(X):
    """Concatenates a bunch of images into a big matrix for plotting purposes."""
    nc, h , w, _ = X.shape
    X = X.reshape(nc, h, w)
    n = np.ceil(np.sqrt(nc)).astype("int8")
    img = np.zeros((n*w,n*h))
    x = 0
    y = 0
    for example in range(nc):
        img[x*w:(x+1)*w,y*h:(y+1)*h] = X[example]
        y += 1
        if y >= n:
            y = 0
            x += 1
    return img

In [ ]:
def plot_oneshot_task(pairs):
    fig,(ax1,ax2) = plt.subplots(nrows=1, ncols=2)
    ax1.matshow(pairs[0][0].reshape(105,105), cmap='gray')
    img = concat_images(pairs[1])
    ax1.get_yaxis().set_visible(False)
    ax1.get_xaxis().set_visible(False)
    ax2.matshow(img,cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
# Example of concat image visualization
pairs, targets = make_oneshot_task(16,"train","Sanskrit")
plot_oneshot_task(pairs)

In [ ]:
fig,ax = plt.subplots(1)
ax.plot(ways, val_accs, "m", label="Siamese(val set)")
ax.plot(ways, train_accs, "y", label="Siamese(train set)")
plt.plot(ways, nn_accs, label="Nearest neighbour")

ax.plot(ways, 100.0/ways, "g", label="Random guessing")
plt.xlabel("Number of possible classes in one-shot tasks")
plt.ylabel("% Accuracy")
plt.title("Omiglot One-Shot Learning Performance of a Siamese Network")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
inputs,targets = make_oneshot_task(20, "val", 'Oriya')
plt.show()

plot_oneshot_task(inputs)